In [39]:
import mysql.connector
from cryptography.fernet import Fernet

In [40]:
conn = mysql.connector.connect(
    user='root',
    password='Qadeerh03',
    host='127.0.0.1',
    database='ecms',
    port=3306
)

In [41]:
# Generate and save this key securely
key = Fernet.generate_key()
cipher_suite = Fernet(key)

In [42]:
# Load your previously saved key
def load_key():
    return open('secure.key', 'rb').read()

In [43]:
# Encrypt the msg
def encrypt_message(message, key):
    return Fernet(key).encrypt(message.encode())

In [44]:
# Key will stored used later for decryption
with open('secure.key', 'wb') as key_file:
    key_file.write(key)

In [45]:
# Create a cursor object
cursor = conn.cursor()

In [46]:
# Function to format contact numbers
def format_contact_number(number):
    if number.startswith('0'):
        number = '+353' + number[1:]
    return number

In [47]:
# Function to insert to create new patient profile
def create_patient(name, dob, contact_number, email_address, home_address, next_of_kin_name, emergency_contact_number, next_of_kin_home_address, emergency_email_address):
    key = load_key()
    try:
        # Format the contact number and the emergency contact number
        formatted_contact_number = format_contact_number(contact_number)
        formatted_emergency_contact_number = format_contact_number(emergency_contact_number)

        # Encrypt data such as (Contact Number, Home Address, Email Address, Next of Kin Name, Emergency Contact Number, Next of Kin Home Address
        encrypted_contact_number = encrypt_message(formatted_contact_number, key)
        encrypted_email_address = encrypt_message(email_address, key)
        encrypted_home_address = encrypt_message(home_address, key)
        encrypted_next_of_kin_name = encrypt_message(next_of_kin_name, key)
        encrypted_emergency_contact_number = encrypt_message(formatted_emergency_contact_number, key)
        encrypted_next_of_kin_home_address = encrypt_message(next_of_kin_home_address, key)
        encrypted_emergency_email_address = encrypt_message(emergency_email_address, key)

        # SQL query to insert a new record into the patient_profile table
        create_patient_profile = '''
        INSERT INTO patient_profile (Name, DOB, Contact_Number, Email_Address, Home_Address, Next_Of_Kin_Name, Emergency_Contact_Number, Next_Of_Kin_Home_Address, Emergency_Email_Address)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
        cursor.execute(create_patient_profile, (name, dob, encrypted_contact_number, encrypted_email_address, encrypted_home_address, encrypted_next_of_kin_name, encrypted_emergency_contact_number, encrypted_next_of_kin_home_address, encrypted_emergency_email_address))
        # Commit to the database
        conn.commit()
        print("Patient record created.")
    # If creating the patient fails throw error
    except mysql.connector.Error as e:
        print("Error occurred:", e)
    finally:
        cursor.close()
        conn.close()

In [48]:
# Dummy Data
create_patient(
    name='John Doe',
    dob='2024-01-01',
    contact_number='0123456789',
    email_address='johndoe@gmail.com',
    home_address='1234 Street Dublin',
    next_of_kin_name='Jane Doe',
    emergency_contact_number='0987654321',
    next_of_kin_home_address='5678 Street Dublin',
    emergency_email_address='janedoe@gmail.com'
)

Patient record created.
